In [1]:
import math
import argparse
import sys
import numpy as np

import torch
from torch import nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable, Function

## Define Module

In [6]:
# class GenWeights(Function):
#     def __init__(self, ops, in_channels, out_channels, kernel_size, groups=1):
#         self.ops = ops
#         self.in_channels = in_channels
#         self.out_channels = out_channels
#         self.kernel_size = kernel_size
#         self.groups = groups
#         self.ncoeff = list(self.ops.size())[-1]
        
#     def forward(self, coeffs):
#         w = torch.sum(torch.mul(coeffs.view(self.out_channels, self.in_channels, 1, 1, self.ncoeff), self.ops), 
#                       dim=-1)
#         weight = Variable(w, requires_grad=True)
#         return weight

#     def backward(self, weight_grad):
#         return torch.mul(weight_grad.unsqueeze(-1), self.ops).sum(2).sum(2)

class FdConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, degree=2, stride=1, padding=0, dilation=1, groups=1, bias=True):
        assert degree in [0,1,2]
        super(FdConv2d, self).__init__()
        if in_channels % groups != 0:
            raise ValueError('in_channels must be divisible by groups')
        if out_channels % groups != 0:
            raise ValueError('out_channels must be divisible by groups')
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.degree = degree
        self.kernel_size = (3, 3)
        self.stride = stride
        self.padding = padding
        self.dilation = dilation
        self.groups = groups
        if bias:
            self.bias = Parameter(torch.Tensor(out_channels))
        else:
            self.register_parameter('bias', None)
#         self.ncoeff = int((self.degree + 1) * (self.degree + 2) / 2)
        self.ncoeff = 4
        self.coeffs = torch.Tensor(out_channels, in_channels // groups, self.ncoeff)
        if torch.cuda.is_available():
            self.coeffs = self.coeffs.cuda()
        self.coeffs = Parameter(self.coeffs)
        self.set_coeffs()
        self.ops = self.get_diff_operators()
        
    def set_coeffs(self):
        n = self.in_channels * self.ncoeff
        stdv = 1. / math.sqrt(n)
        self.coeffs.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)
            
#     def get_diff_operators(self):
#         operators = []
#         # add degree zero
#         operators.append([[0,0,0],[0,1,0],[0,0,0]])
#         # add degree one
#         if self.degree > 0:
#             operators.append([[0,0,0],[-.5,0,.5],[0,0,0]])
#             operators.append([[0,.5,0],[0,0,0],[0,-.5,0]])
#         # add degree two
#         if self.degree > 1:
#             operators.append([[-.25,0,.25],[0,0,0],[.25,0,-.25]])
#             operators.append([[0,0,0],[1,-2,1],[0,0,0]])
#             operators.append([[0,1,0],[0,-2,0],[0,1,0]])
# #         operators.append([[1,0,0],[0,0,0],[0,0,0]])
# #         operators.append([[0,1,0],[0,0,0],[0,0,0]])
# #         operators.append([[0,0,1],[0,0,0],[0,0,0]])
# #         operators.append([[0,0,0],[1,0,0],[0,0,0]])
# #         operators.append([[0,0,0],[0,1,0],[0,0,0]])
# #         operators.append([[0,0,0],[0,0,1],[0,0,0]])
# #         operators.append([[0,0,0],[0,0,0],[1,0,0]])
# #         operators.append([[0,0,0],[0,0,0],[0,1,0]])
# #         operators.append([[0,0,0],[0,0,0],[0,0,1]])
#         ops = torch.tensor(operators, dtype=torch.float)
#         ops = ops.permute(1, 2, 0)  # shape (3, 3, ncoeff)
#         if torch.cuda.is_available():
#             ops = ops.cuda()
#         return Variable(ops)

#     def get_diff_operators(self):
#         ops = torch.Tensor(3, 3, self.ncoeff)
#         ops.data.uniform_(-1, 1)
# #         ops = torch.zeros([3, 3, self.ncoeff])
#         if torch.cuda.is_available():
#             ops = ops.cuda()
#         return Variable(ops)

    def get_diff_operators(self):
        operators = []
        operators.append([[0,0,0],[0,1,0],[0,0,0]])
        operators.append([[0,0,0],[-0.5,0,0.5],[0,0,0]])
        operators.append([[0,0.5,0],[0,0,0],[0,-0.5,0]])
#         operators.append([[0,1,0], [1,-4,1], [0,1,0]])
        operators.append([[0,.5,0], [.5,-2,.5], [0,.5,0]])
        ops = torch.tensor(operators, dtype=torch.float)
        ops = ops.permute(1, 2, 0)  # shape (3, 3, ncoeff)
        if torch.cuda.is_available():
            ops = ops.cuda()
        return Variable(ops)
        
    def forward(self, input):
        self.weight = torch.sum(torch.mul(self.coeffs.view(self.out_channels, self.in_channels, 1, 1, self.ncoeff), self.ops), 
                      dim=-1)
        return F.conv2d(input, self.weight, self.bias, self.stride,
                        self.padding, self.dilation, self.groups)

## Define Model

In [7]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.b = 32
#         self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
#         self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
#         self.conv1 = nn.Conv2d(1, self.b, kernel_size=3, padding=1)
#         self.conv2 = nn.Conv2d(self.b, 2*self.b, kernel_size=3, padding=1)
        self.conv1 = FdConv2d(1, self.b, degree=2, padding=1)
        self.conv2 = FdConv2d(self.b, 2*self.b, degree=2, padding=1)
        self.csize = 2*self.b*7*7
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(self.csize, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, self.csize)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)
    
# class Net(nn.Module):
#     def __init__(self):
#         super(Net, self).__init__()
#         self.csize = 20*7*7
# #         self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
# #         self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
# #         self.conv1 = nn.Conv2d(1, 10, kernel_size=3, padding=1)
# #         self.conv2 = nn.Conv2d(10, 20, kernel_size=3, padding=1)
#         self.conv1 = FdConv2d(1, 10, degree=1, padding=1)
#         self.conv2 = FdConv2d(10, 10, degree=1, padding=1)
#         self.conv3 = FdConv2d(10, 20, degree=1, padding=1)
#         self.conv4 = FdConv2d(20, 20, degree=1, padding=1)
#         self.conv4_drop = nn.Dropout2d()
#         self.fc1 = nn.Linear(self.csize, 50)
#         self.fc2 = nn.Linear(50, 10)

#     def forward(self, x):
#         x = F.relu(self.conv1(x))
#         x = F.relu(F.max_pool2d(self.conv2(x), 2))
#         x = F.relu(self.conv3(x))
#         x = F.relu(F.max_pool2d(self.conv4_drop(self.conv4(x)), 2))
#         x = x.view(-1, self.csize)
#         x = F.relu(self.fc1(x))
#         x = F.dropout(x, training=self.training)
#         x = self.fc2(x)
#         return F.log_softmax(x, dim=1)

## Train and Test

In [8]:
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            sys.stdout.write('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f} \r'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            sys.stdout.flush()

def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, size_average=False).item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%) \r'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

## Run

In [9]:
# Training settings
parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
parser.add_argument('--batch-size', type=int, default=64, metavar='N',
                    help='input batch size for training (default: 64)')
parser.add_argument('--test-batch-size', type=int, default=1000, metavar='N',
                    help='input batch size for testing (default: 1000)')
parser.add_argument('--epochs', type=int, default=50, metavar='N',
                    help='number of epochs to train (default: 10)')
parser.add_argument('--lr', type=float, default=1e-2, metavar='LR',
                    help='learning rate (default: 0.01)')
parser.add_argument('--momentum', type=float, default=0.5, metavar='M',
                    help='SGD momentum (default: 0.5)')
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='disables CUDA training')
parser.add_argument('--seed', type=int, default=1, metavar='S',
                    help='random seed (default: 1)')
parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                    help='how many batches to wait before logging training status')
args = parser.parse_args('')
use_cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.test_batch_size, shuffle=True, **kwargs)

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)
# optimizer = optim.Adam(model.parameters(), lr=args.lr)

for epoch in range(1, args.epochs + 1):
    train(args, model, device, train_loader, optimizer, epoch)
    test(args, model, device, test_loader)

Train Epoch: 1 [59520/60000 (99%)]	Loss: 0.267019 
Test set: Average loss: 0.2231, Accuracy: 9333/10000 (93%) 
Train Epoch: 2 [59520/60000 (99%)]	Loss: 0.260088 
Test set: Average loss: 0.1352, Accuracy: 9599/10000 (96%) 
Train Epoch: 3 [59520/60000 (99%)]	Loss: 0.218220 
Test set: Average loss: 0.1040, Accuracy: 9681/10000 (97%) 
Train Epoch: 4 [59520/60000 (99%)]	Loss: 0.152965 
Test set: Average loss: 0.0798, Accuracy: 9745/10000 (97%) 
Train Epoch: 5 [59520/60000 (99%)]	Loss: 0.160363 
Test set: Average loss: 0.0704, Accuracy: 9776/10000 (98%) 
Train Epoch: 6 [59520/60000 (99%)]	Loss: 0.051751 
Test set: Average loss: 0.0641, Accuracy: 9796/10000 (98%) 
Train Epoch: 7 [59520/60000 (99%)]	Loss: 0.036780 
Test set: Average loss: 0.0584, Accuracy: 9805/10000 (98%) 
Train Epoch: 8 [59520/60000 (99%)]	Loss: 0.140186 
Test set: Average loss: 0.0521, Accuracy: 9826/10000 (98%) 
Train Epoch: 9 [59520/60000 (99%)]	Loss: 0.132502 
Test set: Average loss: 0.0495, Accuracy: 9848/10000 (98%) 
T

In [ ]:
print(model)

In [ ]:
params = dict(model.named_parameters())
for k in params.keys():
    print(k, "shape: ", params[k].shape)

In [ ]:
import torch
from torch.autograd import Variable
from torch import nn
from torchviz import make_dot, make_dot_from_trace

for batch_idx, (data, target) in enumerate(train_loader):
    data, target = data.to(device), target.to(device)
    break
output = model(data)
make_dot(output, params=dict(model.named_parameters()))

In [ ]:
import matplotlib.pyplot as plt
oc, ic = 0, 0
w1 = model.conv1.weight.cpu().detach().numpy().reshape(-1, 3, 3)
w2 = model.conv2.weight.cpu().detach().numpy().reshape(-1, 3, 3)
c1 = model.conv1.coeffs.cpu().detach().numpy().reshape(w1.shape[0], -1)
c2 = model.conv2.coeffs.cpu().detach().numpy().reshape(w2.shape[0], -1)
c = w1
columns = 2
rows = 5
fig = plt.figure(figsize=(5,15))
for i in range(columns*rows):
    img = c[i]
    fig.add_subplot(rows, columns, i+1)
    plt.imshow(img)
    plt.colorbar()
plt.show()
# print(c1.shape)
# print(c2.shape)
# plt.figure(f
# plt.imshow(np.absolute(c1))
# plt.colorbar()
# plt.show()

In [ ]:
plt.figure()
plt.imshow((c1**2))
plt.colorbar()
plt.show()

In [ ]:
plt.figure()
plt.bar(np.arange(6), np.mean(np.absolute(c1**2), axis=0))
plt.show()
plt.figure()
plt.bar(np.arange(6), np.mean(np.absolute(c2**2), axis=0))
plt.show()
